In [1]:
%load_ext autoreload
%load_ext autotime

time: 398 µs (started: 2023-01-25 23:12:46 -08:00)


In [2]:
%run init.ipynb

time: 6.55 s (started: 2023-01-25 23:12:46 -08:00)


In [3]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')

import dask
import dask.array as da
import dask.bag as db
import dask.dataframe as dd
import json
import math
import numpy as np
import os
import pandas as pd
# import pyarrow as pa
from dask import compute, delayed
from dask.dataframe import DataFrame
from dask.distributed import Client, LocalCluster, fire_and_forget, wait, worker_client
from vani.utils.file_utils import ensure_dir
from vani.utils.json_encoders import NpEncoder
from vani.utils.logger import create_logger, format_log

from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

time: 958 ms (started: 2023-01-25 23:12:52 -08:00)


In [4]:
client

Connection method: Cluster object,Cluster type: dask_jobqueue.LSFCluster
Dashboard: http://192.168.66.201:8788/status,
Dashboard: http://192.168.66.201:8788/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://192.168.66.201:37621,Workers: 0
Dashboard: http://192.168.66.201:8788/status,Total threads: 0
Started: Just now,Total memory: 0 B


time: 86.1 ms (started: 2023-01-25 23:12:53 -08:00)


In [5]:
import json

def load_global_min_max(log_dir: str):
    with open(f"{log_dir}/global.json") as file:
        global_min_max = json.load(file)
    return global_min_max

time: 1.25 ms (started: 2023-01-25 23:12:53 -08:00)


In [6]:
from vani.utils.dask_agg import nunique
import copy
import dask.dataframe as dd
import numpy as np
import pandas as pd
from dask import compute


XFER_SIZE_BINS = [
    -np.inf,
    4 * 1024.0,
    16 * 1024.0,
    64 * 1024.0,
    256 * 1024.0,
    1 * 1024.0 * 1024.0,
    4 * 1024.0 * 1024.0,
    16 * 1024.0 * 1024.0,
    64 * 1024.0 * 1024.0,
    np.inf
]
XFER_SIZE_BIN_LABELS = [
    '<4KB',
    '~16KB',
    '~64KB',
    '~256KB',
    '~1MB',
    '~4MB',
    '~16MB',
    '~64MB',
    '>64MB'
]
XFER_SIZE_BIN_NAMES = [
    '<4KB',
    '4KB',
    '16KB',
    '64KB',
    '256KB',
    '1MB',
    '4MB',
    '16MB',
    '64MB',
    '>64MB'
]


def compute_general_stats(ddf: dd.DataFrame):
    return ddf \
        .groupby(['io_cat']) \
        .agg({
            'acc_pat': [min, max],
            'app': [nunique()],
            'duration': [sum],
            'file_id': [nunique()],
            'hostname': [nunique()],
            'index': ['count'],
            'proc_id': [nunique()],
            'rank': [nunique()],
            'size': [min, max, 'mean', sum],
        })


def compute_xfer_size_stats(ddf: dd.DataFrame):
    return ddf[ddf['io_cat'].isin([1, 2])] \
        .map_partitions(lambda df: df.assign(xfer=pd.cut(df['size'], XFER_SIZE_BINS, right=True))) \
        .groupby(['io_cat', 'xfer']) \
        .agg({
            'duration': [sum],
            'size': [sum],
            'index': ['count']
        })


def compute_hlm(ddf: dd.DataFrame, group_by: str, metric='duration', **kwargs):
    return ddf \
        .groupby([group_by]) \
        .agg({metric: [sum]}) \
        .sort_values((metric, 'sum'), ascending=False) \
        .compute()


def compute_hlm2(ddf: dd.DataFrame, group_by: str, filter_by: str, filter_indices: list, metric='duration'):
    return ddf[ddf[filter_by].isin(filter_indices)] \
        .groupby([group_by]) \
        .agg({metric: [sum]}) \
        .sort_values((metric, 'sum'), ascending=False) \
        .compute()


def compute_llc(ddf: dd.DataFrame, group_by: str, filter_by: str, filter_indices: list,):
    return ddf[ddf[filter_by].isin(filter_indices)] \
        .groupby([group_by]) \
        .agg({
            'acc_pat': [min, max],
            'app': ['nunique'],
            'duration': [sum],
            'file_id': [min, 'nunique'],
            'hostname': ['nunique'],
            'index': ['count'],
            'io_cat': [min, max],
            'proc_id': [min, 'nunique'],
            'rank': [min, max, 'nunique'],
            'size': [min, max, 'mean', sum],
        })


def filter_delta(df: pd.DataFrame, delta: float, metric='duration'):
    df[metric, 'csp'] = df[metric, 'sum'].cumsum() / df[metric, 'sum'].sum()
    df[metric, 'delta'] = df[metric, 'csp'].diff().fillna(df[metric, 'csp'])
    df_filtered = df[df[metric, 'delta'] > delta]
    df_filtered = df_filtered.reindex(sorted(df_filtered.columns), axis=1)
    return df_filtered

def filter_delta_persisted(ddf: dd.DataFrame, delta: float, metric='duration'):
    def set_csp_delta(df):
        df[metric, 'csp'] = df[metric, 'sum'].cumsum() / df[metric, 'sum'].sum()
        df[metric, 'delta'] = df[metric, 'csp'].diff().fillna(df[metric, 'csp'])
        return df
        
    return ddf.map_partitions(set_csp_delta)[ddf[metric, 'delta'] > delta].reindex(sorted(df_filtered.columns), axis=1)
#     df[metric, 'csp'] = df[metric, 'sum'].cumsum() / df[metric, 'sum'].sum()
#     df[metric, 'delta'] = df[metric, 'csp'].diff().fillna(df[metric, 'csp'])
#     df_filtered = df[df[metric, 'delta'] > delta]
#     df_filtered = df_filtered.reindex(sorted(df_filtered.columns), axis=1)
#     return df_filtered


time: 8.32 ms (started: 2023-01-25 23:12:53 -08:00)


In [7]:
DELTA = 0.0001

time: 601 µs (started: 2023-01-25 23:12:53 -08:00)


In [6]:
import logging
import os
import socket
from dask.distributed import Client, LocalCluster, get_task_stream, progress, wait
from dask_jobqueue import LSFCluster
from time import perf_counter, sleep

node_memory = 1600 # node memory in GB
n_workers_per_node = 32 # number of worker processes per node
n_threads_per_worker = 16
worker_time = 120 # job time per node for worker
worker_queue = "pdebug" # queue to be used per worker

def init_cluster(cluster_key: str, port: int):    
    import socket
    host = socket.gethostname()
    dashboard_address = f'{socket.gethostname()}:{port}'
    memory = '{}GB'.format(node_memory/n_workers_per_node)
    job_extra = ['-nnodes 1', 
                 '-G asccasc', 
                 '-q {}'.format(worker_queue), 
                 '-W {}'.format(worker_time)]
    local_directory=f"/var/tmp/dask/{cluster_key}"
    cluster = LSFCluster(cores = n_workers_per_node*n_threads_per_worker,processes=n_workers_per_node, memory='{}GB'.format(node_memory), 
                         header_skip=['-n ','-R','-M', '-P', '-W 00:30'], 
                         job_extra = job_extra, 
                         death_timeout=worker_time*60,
                         local_directory =f"{local_directory}",
                         use_stdin=True, 
                         scheduler_options={"port": 0,"host":host, "dashboard_address": f':{port}'})
    return cluster

FILTER_GROUPS = ['trange', 'file_id', 'proc_id']


clusters = {}
port = 8555
for fg in FILTER_GROUPS:
    clusters[fg] = init_cluster(fg, port)
    port = port + 1
clusters

{'trange': LSFCluster(02cae650, 'tcp://192.168.66.200:42519', workers=0, threads=0, memory=0 B),
 'file_id': LSFCluster(a5711d35, 'tcp://192.168.66.200:32803', workers=0, threads=0, memory=0 B),
 'proc_id': LSFCluster(97a1e56d, 'tcp://192.168.66.200:43105', workers=0, threads=0, memory=0 B)}

time: 1.16 s (started: 2023-01-04 07:55:45 -08:00)


In [7]:
for cluster_key in clusters:
    clusters[cluster_key].scale(n_workers_per_node*2)

time: 3.51 ms (started: 2023-01-04 07:55:46 -08:00)


In [8]:
clients = {}
for cluster_key in clusters:
    clients[cluster_key] = Client(clusters[cluster_key])

time: 48.9 ms (started: 2023-01-04 07:55:47 -08:00)


In [9]:
clients

{'trange': <Client: 'tcp://192.168.66.200:42519' processes=0 threads=0, memory=0 B>,
 'file_id': <Client: 'tcp://192.168.66.200:32803' processes=0 threads=0, memory=0 B>,
 'proc_id': <Client: 'tcp://192.168.66.200:43105' processes=0 threads=0, memory=0 B>}

time: 5.29 ms (started: 2023-01-04 07:55:49 -08:00)


In [11]:
async def compute_metrics_async(log_dir: str, filter_group: str):
    print(log_dir, filter_group)
    with clients[filter_group].as_current():
        print(dask.distributed.get_client())
        # Read logs
        ddf = dd.read_parquet(f"{log_dir}/*.parquet")
        # Set trange
        tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
        trange = np.arange(tmid_min, tmid_max, 1e7)
        ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
        # Compute HLM
        hlm = compute_hlm(ddf=ddf, group_by=filter_group)
        # Filter HLM
        hlm_filtered = filter_delta(df=hlm, delta=DELTA)

        return hlm_filtered

def compute_high_level_metrics(log_dir: str, filter_group: str):
    print(log_dir, filter_group)
    with clients[filter_group].as_current():
        print(dask.distributed.get_client())
        # Read logs
#         ddf = dd.read_parquet(f"{log_dir}/*.parquet")
        columns = list(set(map(lambda x: x.replace('trange', 'tmid'), [filter_group, 'duration', 'tmid'])))
        ddf = dd.read_parquet(f"{log_dir}/*.parquet", columns=columns)
        # Set trange
        tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
        trange = np.arange(tmid_min, tmid_max, 1e7)
        ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
        # Compute HLM
        hlm = compute_hlm(ddf=ddf, group_by=filter_group)
        # Filter HLM
        hlm_filtered = filter_delta(df=hlm, delta=DELTA)

#         hlm_filtered_file_id = compute_hlm2(ddf=ddf, group_by='file_id', filter_by=filter_group, filter_indices=hlm_filtered.index)
        
#         hlm_filtered_file_id_filtered = filter_delta(df=hlm_filtered_file_id, delta=DELTA)

        return filter_group, hlm_filtered

    
def compute_permutations(log_dir: str, filter_perm: tuple, hlm: pd.DataFrame):
    filter_group, cross_group = filter_perm
    print(log_dir, filter_group, cross_group)
    with clients[filter_group].as_current():
        print(dask.distributed.get_client())
        # Read logs
        columns = list(set(map(lambda x: x.replace('trange', 'tmid'), [filter_group, cross_group, 'duration', 'tmid'])))
        ddf = dd.read_parquet(f"{log_dir}/*.parquet", columns=columns)
        # Set trange
        tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
        trange = np.arange(tmid_min, tmid_max, 1e7)
        ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
        # Compute HLM
        # hlm = compute_hlm(ddf=ddf, group_by=filter_group)
        # Filter HLM
        # hlm_filtered = filter_delta(df=hlm, delta=DELTA)

        hlm_filtered_file_id = compute_hlm2(ddf=ddf, group_by=cross_group, filter_by=filter_group, filter_indices=hlm.index)
        
        hlm_filtered_file_id_filtered = filter_delta(df=hlm_filtered_file_id, delta=DELTA)

        return hlm_filtered_file_id_filtered

time: 6.33 ms (started: 2023-01-04 07:56:00 -08:00)


In [12]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from itertools import repeat

log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"

results = []
with ThreadPoolExecutor() as executor:
    for filter_group, hlm in executor.map(compute_high_level_metrics, repeat(log_dir), FILTER_GROUPS):
#         filter_group, hlm_filtered = result
        print(filter_group)
#         print(result)
        results.append(hlm)

/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet trange
<Client: 'tcp://192.168.66.200:42519' processes=32 threads=512, memory=1.46 TiB>
/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet file_id
<Client: 'tcp://192.168.66.200:32803' processes=32 threads=512, memory=1.46 TiB>
/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet proc_id
<Client: 'tcp://192.168.66.200:43105' processes=32 threads=512, memory=1.46 TiB>
trange
file_id
proc_id
time: 29.6 s (started: 2023-01-04 07:56:04 -08:00)


In [ ]:
from itertools import chain, permutations

N_PERMUTATIONS = 2

results2 = []
with ThreadPoolExecutor() as executor:
    for res in executor.map(
        compute_permutations, 
        repeat(log_dir), 
        permutations(FILTER_GROUPS, N_PERMUTATIONS), 
        chain.from_iterable(repeat(result, N_PERMUTATIONS) for result in results)
    ):
        #         filter_group, hlm_filtered = result
        print(res)
        results2.append(res)
        pass
#         print(result)
        # results2.append(hlm2)


In [38]:
results2

[                      duration                        
                            csp     delta           sum
 file_id                                               
  113104469769545699   0.387302  0.387302  48777.953125
  1917887208199751214  0.399103  0.011801   1486.222412
  1917887209186244162  0.407242  0.008139   1025.085693
  1917887210832504031  0.415351  0.008108   1021.207886
  1917887209973776488  0.423086  0.007735    974.176453
 ...                        ...       ...           ...
 -7417831731040427752  0.788386  0.000101     12.681411
  8686466593926285568  0.788486  0.000101     12.667913
  818293175134167005   0.788587  0.000101     12.667041
 -6427059915332952398  0.788687  0.000100     12.631663
 -997670665515561906   0.788788  0.000100     12.618176
 
 [1482 rows x 3 columns],
                       duration                        
                            csp     delta           sum
 file_id                                               
  113104469769545699

time: 24.6 ms (started: 2023-01-03 22:29:24 -08:00)


In [14]:
clients

{'trange': <Client: 'tcp://192.168.66.200:33549' processes=64 threads=1024, memory=2.91 TiB>,
 'file_id': <Client: 'tcp://192.168.66.200:35887' processes=64 threads=1024, memory=2.91 TiB>,
 'proc_id': <Client: 'tcp://192.168.66.200:34735' processes=64 threads=1024, memory=2.91 TiB>}

time: 6.39 ms (started: 2023-01-03 22:10:17 -08:00)


In [8]:
!du -sh /p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet

27G	/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet
time: 161 ms (started: 2023-01-21 18:19:10 -08:00)


In [10]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
df = pd.read_parquet(f"{log_dir}/")
df

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
0,1,53431,0,315008,0,3,0.237784,0.237786,getcwd,0,lassen72,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000002,2377848,113104469769545699,7523492661920059012
1,2,53431,0,315008,0,3,0.250517,0.250523,closedir,0,lassen72,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000006,2505197,113104469769545699,7523492661920059012
2,3,53431,0,315008,0,3,0.250669,0.250673,opendir,0,lassen72,pegasus-mpi-cluster,/g/g92/haridev/.openmpi/components,0,0,0.000000,0.000004,2506708,5197033824288711371,7523492661920059012
3,4,53431,0,315008,0,3,0.291718,0.291721,getcwd,0,lassen72,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000003,2917192,113104469769545699,7523492661920059012
4,5,53431,0,315008,0,3,0.295864,0.295868,closedir,0,lassen72,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000004,2958657,113104469769545699,7523492661920059012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715248235,10485756,102676,0,31131248,0,1,28.880896,28.880905,read,0,lassen339,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,868.055542,0.000009,288809005,1917887209973776488,3612046420531480651
715248236,10485757,102676,0,31131248,0,1,28.880917,28.880924,read,0,lassen339,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,868.055542,0.000009,288809205,1917887209973776488,3612046420531480651
715248237,10485758,102676,0,31131248,0,1,28.880938,28.880945,read,0,lassen339,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,858.516479,0.000009,288809415,1917887209973776488,3612046420531480651
715248238,10485759,102676,0,31131248,0,1,28.880964,28.880974,read,0,lassen339,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,858.516479,0.000009,288809693,1917887209973776488,3612046420531480651


time: 6min 24s (started: 2023-01-21 18:20:38 -08:00)


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715248240 entries, 0 to 715248239
Data columns (total 20 columns):
 #   Column     Dtype  
---  ------     -----  
 0   index      int64  
 1   proc       int64  
 2   rank       int32  
 3   thread_id  int32  
 4   cat        int32  
 5   io_cat     int32  
 6   tstart     float32
 7   tend       float32
 8   func_id    object 
 9   level      int32  
 10  hostname   object 
 11  app        object 
 12  filename   object 
 13  size       int64  
 14  acc_pat    int32  
 15  bandwidth  float32
 16  duration   float32
 17  tmid       int64  
 18  file_id    int64  
 19  proc_id    int64  
dtypes: float32(4), int32(6), int64(6), object(4)
memory usage: 79.9+ GB
time: 8.51 ms (started: 2023-01-21 18:40:11 -08:00)


In [13]:
df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715248240 entries, 0 to 715248239
Data columns (total 20 columns):
 #   Column     Dtype  
---  ------     -----  
 0   index      int64  
 1   proc       int64  
 2   rank       int32  
 3   thread_id  int32  
 4   cat        int32  
 5   io_cat     int32  
 6   tstart     float32
 7   tend       float32
 8   func_id    object 
 9   level      int32  
 10  hostname   object 
 11  app        object 
 12  filename   object 
 13  size       int64  
 14  acc_pat    int32  
 15  bandwidth  float32
 16  duration   float32
 17  tmid       int64  
 18  file_id    int64  
 19  proc_id    int64  
dtypes: float32(4), int32(6), int64(6), object(4)
memory usage: 271.9 GB
time: 6min 44s (started: 2023-01-21 18:30:29 -08:00)


In [17]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
trange = np.arange(tmid_min, tmid_max, 1e7)
df = df.assign(trange=np.digitize(df['tmid'], trange, right=True))
df

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,...,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id,trange
0,1,53431,0,315008,0,3,0.237784,0.237786,getcwd,0,...,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000002,2377848,113104469769545699,7523492661920059012,1
1,2,53431,0,315008,0,3,0.250517,0.250523,closedir,0,...,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000006,2505197,113104469769545699,7523492661920059012,1
2,3,53431,0,315008,0,3,0.250669,0.250673,opendir,0,...,pegasus-mpi-cluster,/g/g92/haridev/.openmpi/components,0,0,0.000000,0.000004,2506708,5197033824288711371,7523492661920059012,1
3,4,53431,0,315008,0,3,0.291718,0.291721,getcwd,0,...,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000003,2917192,113104469769545699,7523492661920059012,1
4,5,53431,0,315008,0,3,0.295864,0.295868,closedir,0,...,pegasus-mpi-cluster,UNKNOWN,0,0,0.000000,0.000004,2958657,113104469769545699,7523492661920059012,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
715248235,10485756,102676,0,31131248,0,1,28.880896,28.880905,read,0,...,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,868.055542,0.000009,288809005,1917887209973776488,3612046420531480651,29
715248236,10485757,102676,0,31131248,0,1,28.880917,28.880924,read,0,...,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,868.055542,0.000009,288809205,1917887209973776488,3612046420531480651,29
715248237,10485758,102676,0,31131248,0,1,28.880938,28.880945,read,0,...,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,858.516479,0.000009,288809415,1917887209973776488,3612046420531480651,29
715248238,10485759,102676,0,31131248,0,1,28.880964,28.880974,read,0,...,individuals,/p/gpfs1/iopp/temp/1000-genome-haridev/scratch...,8192,0,858.516479,0.000009,288809693,1917887209973776488,3612046420531480651,29


time: 4min 6s (started: 2023-01-21 18:41:55 -08:00)


In [21]:
df['acc_pat'] = df['acc_pat'].astype('i1')
df['io_cat'] = df['io_cat'].astype('i1')

all_df = df \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    })

all_df

duration acc_pat      \
                                                             sum     min max   
trange file_id              proc_id             io_cat                         
1      -9205466554323238398 1653728027193903179 3       0.000089       0   0   
       -8827782554772936157 1653728027193903179 3       0.000089       0   0   
       -8744043424653673922 1653728027193903179 3       0.000089       0   0   
       -8242902578254792859 1653728027193903179 3       0.000091       0   0   
       -7858734180709747821 1653728027193903179 3       0.000095       0   0   
...                                                          ...     ...  ..   
3257    1917887211417019558 7523492661920059012 3       0.002786       0   0   
        1917887211422191182 7523492661920059012 3       0.003045       0   0   
        1917887211504003738 7523492661920059012 3       0.002781       0   0   
        1917887211717480448 7523492661920059012 3       0.002475       0   0   
        1917887211780528149 7523492661920059012 3       0.002977       0   0   

                                                       size         index  
                                                        min max sum count  
trange file_id              proc_id             io_cat                     
1      -9205466554323238398 1653728027193903179 3         0   0   0     1  
       -8827782554772936157 1653728027193903179 3         0   0   0     1  
       -8744043424653673922 1653728027193903179 3         0   0   0     1  
       -8242902578254792859 1653728027193903179 3         0   0   0     1  
       -7858734180709747821 1653728027193903179 3         0   0   0     1  
...                                                     ...  ..  ..   ...  
3257    1917887211417019558 7523492661920059012 3         0   0   0     1  
        1917887211422191182 7523492661920059012 3         0   0   0     1  
        1917887211504003738 7523492661920059012 3         0   0   0     1  
        1917887211717480448 7523492661920059012 3         0   0   0     1  
        1917887211780528149 7523492661920059012 3         0   0   0     1  

[54655536 rows x 7 columns]

time: 4min 34s (started: 2023-01-21 19:33:32 -08:00)


In [22]:
all_df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54655536 entries, (1, -9205466554323238398, 1653728027193903179, 3) to (3257, 1917887211780528149, 7523492661920059012, 3)
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   (duration, sum)  float32
 1   (acc_pat, min)   int8   
 2   (acc_pat, max)   int8   
 3   (size, min)      int64  
 4   (size, max)      int64  
 5   (size, sum)      int64  
 6   (index, count)   int64  
dtypes: float32(1), int64(4), int8(2)
memory usage: 2.6 GB
time: 1.33 s (started: 2023-01-21 19:38:07 -08:00)


In [19]:
trange_df = all_df \
    .groupby('trange') \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False)

trange_df

duration acc_pat           size                          index
                sum     min max        min        max         sum   count
trange                                                                   
109     1721.090942       0  40  693697375  758505618  3858948395  460578
111     1690.369751       0  39  768581010  822274763  5405174646  561200
108     1677.583862       0  38  694670290  730404524  4355503898  438780
107     1543.197266       0  37  589040865  617570274  4356702262  481463
113     1475.074829       0  36  546397583  605311855  5001712675  563823
...             ...     ...  ..        ...        ...         ...     ...
3087       0.000000       0   0          0          0           0       1
3084       0.000000       0   0          0          0           0       1
3080       0.000000       0   0          0          0           0       1
2839       0.000000       0   0          0          0           0       1
3107       0.000000       0   0          0          0           0       1

[2589 rows x 7 columns]

time: 4.65 s (started: 2023-01-21 19:00:38 -08:00)


In [8]:
log_dir = "/p/gpfs1/iopp/recorder_app_logs/genome_pegasus/nodes-32/_parquet"
ddf = dd.read_parquet(f"{log_dir}/*.parquet")
ddf 

,index,proc,rank,thread_id,cat,io_cat,tstart,tend,func_id,level,hostname,app,filename,size,acc_pat,bandwidth,duration,tmid,file_id,proc_id
npartitions=698,,,,,,,,,,,,,,,,,,,,
,int64,int64,int32,int32,int32,int32,float32,float32,object,int32,object,object,object,int64,int32,float32,float32,int64,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


time: 978 ms (started: 2023-01-25 23:13:01 -08:00)


In [9]:
tmid_min, tmid_max = load_global_min_max(log_dir=log_dir)['tmid']
trange = np.arange(tmid_min, tmid_max, 1e7)
# ddf = ddf.map_partitions(lambda df: df.assign(trange=pd.cut(df['tmid'], trange, right=True)))
ddf = ddf.map_partitions(lambda df: df.assign(trange=np.digitize(df['tmid'], trange, right=True)))

time: 19.2 ms (started: 2023-01-25 23:13:02 -08:00)


In [11]:
%%timeit -r 10

trange_agg = ddf \
    .groupby(['trange']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()
trange_agg.sort_values(('duration', 'sum'), ascending=False)  

13.4 s ± 568 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 30s (started: 2023-01-11 15:28:26 -08:00)


In [12]:
%%timeit -r 10

trange_agg2 = ddf \
    .groupby(['trange', 'io_cat']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()
trange_agg2.sort_values(('duration', 'sum'), ascending=False)  

14 s ± 292 ms per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 2min 34s (started: 2023-01-11 15:30:57 -08:00)


In [10]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        # 'size':[sum],
#         'index':['count']
    }) \
    .compute()

1min 26s ± 701 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 8min 41s (started: 2023-01-18 16:09:47 -08:00)


In [11]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'size':[sum],
#         'size': [min, max, 'mean', sum],
#         'index':['count']
    }) \
    .compute()

1min 34s ± 660 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 9min 27s (started: 2023-01-18 16:18:29 -08:00)


In [13]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'size':[sum],
#         'size': [min, max, 'mean', sum],
        'index':['count']
    }) \
    .compute()

1min 44s ± 757 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 10min 26s (started: 2023-01-18 16:43:30 -08:00)


In [15]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
#         'size':[sum],
        'size': ['mean', sum],
        'index':['count']
    }) \
    .compute()

1min 54s ± 920 ms per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 11min 25s (started: 2023-01-18 17:36:25 -08:00)


In [16]:
%%timeit -r 5

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
#         'size':[sum],
        'size': [min, max, 'mean', sum],
        'index':['count']
    }) \
    .compute()

2min 14s ± 1.32 s per loop (mean ± std. dev. of 5 runs, 1 loop each)
time: 13min 23s (started: 2023-01-18 17:48:35 -08:00)


In [11]:
%%timeit -r 10

ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

2min 28s ± 6.59 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 27min 7s (started: 2023-01-19 20:27:37 -08:00)


In [13]:
%%timeit -r 10

ddf['acc_pat'] = ddf['acc_pat'].astype('i4')
ddf['io_cat'] = ddf['io_cat'].astype('i4')

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

3min 1s ± 6.43 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 32min 52s (started: 2023-01-19 21:03:58 -08:00)


In [13]:
%%timeit -r 10

ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat'], dropna=False, observed=True) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

2min 26s ± 2.13 s per loop (mean ± std. dev. of 10 runs, 1 loop each)
time: 26min 56s (started: 2023-01-19 13:33:38 -08:00)


In [14]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i4')
ddf['io_cat'] = ddf['io_cat'].astype('i4')

all_agg = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

time: 3min 16s (started: 2023-01-19 21:37:16 -08:00)


In [15]:
all_agg.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54655536 entries, (1, 113104469769545699, 109922137388615755, 3) to (2050, 113104469769545699, 7523492662002962052, 0)
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   (duration, sum)  float32
 1   (acc_pat, min)   int32  
 2   (acc_pat, max)   int32  
 3   (size, min)      int64  
 4   (size, max)      int64  
 5   (size, sum)      int64  
 6   (index, count)   int64  
dtypes: float32(1), int32(2), int64(4)
memory usage: 3.4 GB
time: 1.35 s (started: 2023-01-19 21:40:33 -08:00)


In [10]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .compute()

time: 2min 19s (started: 2023-01-25 22:16:32 -08:00)


In [10]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg_to_persist = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .reset_index() \
    .repartition('100MB')

all_agg_to_persist.columns = ['_'.join(tup).rstrip('_') for tup in all_agg_to_persist.columns.values]
all_agg_to_persist.to_parquet(f"{log_dir}/agg/")

[None]

time: 3min 52s (started: 2023-01-25 23:13:37 -08:00)


In [11]:
all_agg_from_persist = dd.read_parquet(f"{log_dir}/agg/*.parquet")
all_agg_from_persist

,trange,file_id,proc_id,io_cat,duration_sum,acc_pat_min,acc_pat_max,size_min,size_max,size_sum,index_count
npartitions=39,,,,,,,,,,,
0,int64,int64,int64,int64,float32,int8,int8,int64,int64,int64,int64
1401424,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...
53254112,...,...,...,...,...,...,...,...,...,...,...
54655535,...,...,...,...,...,...,...,...,...,...,...


time: 895 ms (started: 2023-01-25 23:17:29 -08:00)


In [21]:
all_agg_to_persist = all_agg.copy()
all_agg_to_persist.columns = ['_'.join(tup).rstrip('_') for tup in all_agg_to_persist.columns.values]
# all_agg_to_persist.to_parquet(f"{log_dir}/agg")
all_agg_to_persist.columns

Index(['duration_sum', 'acc_pat_min', 'acc_pat_max', 'size_min', 'size_max',
       'size_sum', 'index_count'],
      dtype='object')

time: 368 ms (started: 2023-01-25 22:32:25 -08:00)


In [17]:
all_agg.info(memory_usage='deep')


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 54655536 entries, (1, 113104469769545699, 109922137388615755, 3) to (2050, 113104469769545699, 7523492662002962052, 0)
Data columns (total 7 columns):
 #   Column           Dtype  
---  ------           -----  
 0   (duration, sum)  float32
 1   (acc_pat, min)   int8   
 2   (acc_pat, max)   int8   
 3   (size, min)      int64  
 4   (size, max)      int64  
 5   (size, sum)      int64  
 6   (index, count)   int64  
dtypes: float32(1), int64(4), int8(2)
memory usage: 3.1 GB
time: 1.34 s (started: 2023-01-19 21:43:45 -08:00)


In [18]:
all_agg.groupby(level='trange').sum().sort_values(('duration', 'sum'), ascending=False)


duration acc_pat             size                          index
                sum     min   max        min        max         sum   count
trange                                                                     
109     1721.090942     0.0  40.0  693697375  758505618  3858948395  460578
111     1690.369751     0.0  39.0  768581010  822274763  5405174646  561200
108     1677.583862     0.0  38.0  694670290  730404524  4355503898  438780
107     1543.197266     0.0  37.0  589040865  617570274  4356702262  481463
113     1475.074829     0.0  36.0  546397583  605311855  5001712675  563823
...             ...     ...   ...        ...        ...         ...     ...
3087       0.000000     0.0   0.0          0          0           0       1
3084       0.000000     0.0   0.0          0          0           0       1
3080       0.000000     0.0   0.0          0          0           0       1
2839       0.000000     0.0   0.0          0          0           0       1
3107       0.000000     0.0   0.0          0          0           0       1

[2589 rows x 7 columns]

time: 3.62 s (started: 2023-01-19 21:44:11 -08:00)


In [27]:
all_agg.reset_index(['file_id', 'proc_id']).groupby(['trange', 'io_cat']).nunique()


file_id proc_id duration acc_pat     size                index
                                   sum     min max  min  max mean  sum count
trange io_cat                                                               
1      1           14     518      525       1   1    3    3    8  170   167
       2          114       1       13       1   1  106  106  106  106     1
       3          263    3870     3466       1   2    2    1    4    2   452
2      1           20     385      379       1   1    4    4    7  243   241
       2         1005       9      200       1   1  527  527  527  527     1
...               ...     ...      ...     ...  ..  ...  ...  ...  ...   ...
3254   3           88       1       88       1   1    1    1    1    1     1
3255   3           64       1       63       1   1    1    1    1    1     1
3256   3           76       1       76       1   1    1    1    1    1     1
3257   0            1       1        1       1   1    1    1    1    1     1
       3           24       1       24       1   1    1    1    1    1     1

[7718 rows x 10 columns]

time: 1min 7s (started: 2023-01-19 19:51:52 -08:00)


In [28]:
all_agg.reset_index(['trange', 'proc_id']).groupby(['file_id']).nunique()


trange proc_id duration acc_pat     size               \
                                         sum     min max  min max mean sum   
file_id                                                                      
-9223371986035283781      1       1        1       1   1    1   1    1   1   
-9223363969247508558      1       1        1       1   1    1   1    1   1   
-9223362727731233329      1       1        1       1   1    1   1    1   1   
-9223362290314498489      1       1        1       1   1    1   1    1   1   
-9223362230599577161      1       1        1       1   1    1   1    1   1   
...                     ...     ...      ...     ...  ..  ...  ..  ...  ..   
 9223361494309915981      1       1        1       1   1    1   1    1   1   
 9223362658809780944      1       1        1       1   1    1   1    1   1   
 9223364712007579008      1       1        1       1   1    1   1    1   1   
 9223368457304021926      1       1        1       1   1    1   1    1   1   
 9223369538921024184      1       1        1       1   1    1   1    1   1   

                     index  
                     count  
file_id                     
-9223371986035283781     1  
-9223363969247508558     1  
-9223362727731233329     1  
-9223362290314498489     1  
-9223362230599577161     1  
...                    ...  
 9223361494309915981     1  
 9223362658809780944     1  
 9223364712007579008     1  
 9223368457304021926     1  
 9223369538921024184     1  

[21267734 rows x 10 columns]

time: 1min 45s (started: 2023-01-19 19:59:05 -08:00)


In [17]:
ddf['acc_pat'] = ddf['acc_pat'].astype('i1')
ddf['io_cat'] = ddf['io_cat'].astype('i1')

all_agg_persisted = ddf \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .agg({
        'duration':[sum],
        'acc_pat': [min, max],
        'size': [min, max, sum],
        'index':['count']
    }) \
    .reset_index() \
    .persist()

time: 109 ms (started: 2023-01-24 21:35:03 -08:00)


In [18]:
all_agg_persisted

Dask DataFrame Structure:
              trange file_id proc_id io_cat duration acc_pat         size                index
                                                 sum     min   max    min    max    sum  count
npartitions=1                                                                                 
               int64   int64   int64  int64  float32    int8  int8  int64  int64  int64  int64
                 ...     ...     ...    ...      ...     ...   ...    ...    ...    ...    ...
Dask Name: reset_index, 1 tasks

time: 41.4 ms (started: 2023-01-24 21:35:05 -08:00)


In [19]:
trange_view_persisted = all_agg_persisted[['trange', 'duration']] \
    .groupby(['trange']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 36.7 ms (started: 2023-01-24 21:35:05 -08:00)


In [20]:
file_id_view_persisted = all_agg_persisted[['file_id', 'duration']] \
    .groupby(['file_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 35 ms (started: 2023-01-24 21:35:06 -08:00)


In [21]:
proc_id_view_persisted = all_agg_persisted[['proc_id', 'duration']] \
    .groupby(['proc_id']) \
    .sum() \
    .sort_values(('duration', 'sum'), ascending=False) \
    .persist()

time: 34 ms (started: 2023-01-24 21:35:07 -08:00)


In [22]:
wait([trange_view_persisted, file_id_view_persisted, proc_id_view_persisted])

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-035c237037e2f63fca4d036c72caac32', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-87c83f348b5e9330e998567f97ce4291', 0)>, <Future: finished, type: pandas.core.frame.DataFrame, key: ('sort_values-ae7723beeb60667f5ef62cde1bb109c8', 0)>}, not_done=set())

time: 2min 5s (started: 2023-01-24 21:35:08 -08:00)


In [24]:
proc_id_view_persisted.head()

,duration
,sum
proc_id,
8712563415903635531,1359.907715
6865659738399443019,1345.294189
6865660408414341195,1343.329102
5225694538768582731,1342.854126
8719823684455368779,1334.649414


time: 25.4 ms (started: 2023-01-24 21:37:51 -08:00)


In [26]:
trange_view_persisted.head()

,duration
,sum
trange,
109,1721.090942
111,1690.369751
108,1677.583862
107,1543.197266
113,1475.074829


time: 29.4 ms (started: 2023-01-24 21:39:01 -08:00)


In [27]:
trange_view_filtered = trange_view_persisted \
    .map_partitions(lambda df: df.assign(csp=df[('duration', 'sum')].cumsum() / df[('duration', 'sum')].sum())) \
    .map_partitions(lambda df: df.assign(delta=df['csp'].diff().fillna(df['csp'])))

trange_view_filtered = trange_view_filtered \
    .loc[trange_view_filtered['delta'] > DELTA] \
    .drop(columns=['csp', 'delta']) \
    .persist()

trange_view_filtered

,duration
,sum
npartitions=1,
,float32
,...


time: 65.6 ms (started: 2023-01-24 21:40:00 -08:00)


In [29]:
trange_view_filtered.head()

,duration
,sum
trange,
109,1721.090942
111,1690.369751
108,1677.583862
107,1543.197266
113,1475.074829


time: 30.6 ms (started: 2023-01-24 21:40:10 -08:00)


In [30]:
all_agg_persisted.index

Dask Index Structure:
npartitions=1
    int64
      ...
dtype: int64
Dask Name: reset_index, 2 tasks

time: 7.68 ms (started: 2023-01-24 21:40:16 -08:00)


In [31]:
trange_view_indices = list(set(trange_view_filtered.index.compute()))
trange_llc = all_agg_persisted[all_agg_persisted['trange'].isin(trange_view_indices)] \
    .groupby(['trange', 'file_id', 'proc_id', 'io_cat']) \
    .sum() \
    .persist()
#     .sort_values(('duration', 'sum'), ascending=False) \
#     .persist()

time: 77.6 ms (started: 2023-01-24 21:41:38 -08:00)


In [32]:
trange_llc

Dask DataFrame Structure:
              duration acc_pat         size                index
                   sum     min   max    min    max    sum  count
npartitions=1                                                   
               float32    int8  int8  int64  int64  int64  int64
                   ...     ...   ...    ...    ...    ...    ...
Dask Name: dataframe-groupby-sum-agg, 1 tasks

time: 33.9 ms (started: 2023-01-24 21:41:41 -08:00)


In [33]:
wait(trange_llc)

DoneAndNotDoneFutures(done={<Future: finished, type: pandas.core.frame.DataFrame, key: ('dataframe-groupby-sum-agg-1dcf0f630d65d8e187e5b8b674fbb1ca', 0)>}, not_done=set())

time: 26.4 s (started: 2023-01-24 21:41:48 -08:00)


In [36]:
trange_llc.count().compute()

duration  sum      25059324
acc_pat   min      25059324
          max      25059324
size      min      25059324
          max      25059324
          sum      25059324
index     count    25059324
dtype: int64

time: 489 ms (started: 2023-01-24 21:42:37 -08:00)


In [39]:
ddf.index.count().compute()

715248240

time: 12.8 s (started: 2023-01-24 21:45:31 -08:00)


In [40]:
25059324 / 715248240

0.03503584154223155

time: 3.74 ms (started: 2023-01-24 21:46:43 -08:00)


In [ ]:
%%timeit -r 10

trange_agg = ddf \
    .groupby(['trange', 'io_cat']) \
    .agg({'duration':[sum]}) \
    .compute() \
    .sort_values(('duration', 'sum'), ascending=False)

In [33]:
trange_agg['duration', 'cumsum'] = trange_agg['duration', 'sum'].cumsum() / trange_agg['duration', 'sum'].sum()
# trange_agg_file['duration', 'delta'] = trange_agg_file['duration', 'cumsum'] / trange_agg_file['duration', 'cumsum'].shift()
trange_agg['duration', 'delta'] = trange_agg['duration', 'cumsum'].diff().fillna(trange_agg['duration', 'cumsum'])
trange_agg['duration', 'per'] = trange_agg.div(trange_agg['duration', 'sum'].sum(), level=0)['duration', 'sum'] * 100
trange_agg_cut =  trange_agg[trange_agg['duration', 'delta'] > DELTA]
trange_agg_cut = trange_agg_cut.reindex(sorted(trange_agg_cut.columns), axis=1)
trange_agg_cut

duration                                    index
                 cumsum     delta       per         sum    count
trange io_cat                                                   
1      3       0.001178  0.001178  0.117756  157.112274  4738612
2      3       0.003216  0.002039  0.203882  272.024048  2187539
3      1       0.004104  0.000888  0.088809  118.490753  9399964
       3       0.006605  0.002500  0.250014  333.574982  1892752
4      3       0.009288  0.002684  0.268370  358.065979  1428528
...                 ...       ...       ...         ...      ...
82     1       0.979731  0.000144  0.014394   19.205080  1840564
83     1       0.979846  0.000115  0.011461   15.291310  1500505
95     1       0.980222  0.000196  0.019592   26.140547  2055562
96     1       0.980393  0.000171  0.017152   22.884914  1739375
1331   3       0.980979  0.000117  0.011681   15.584929    19290

[804 rows x 5 columns]

time: 36.4 ms (started: 2023-01-10 21:03:27 -08:00)


In [43]:
trange_agg_cut_piv = trange_agg_cut.reset_index().pivot(index='trange', columns='io_cat', values=('duration', 'sum')).fillna(0).rename_axis(columns=None)
trange_agg_cut_piv.rename(columns={1:'read', 2: 'write', 3: 'metadata'}, inplace=True)
trange_agg_cut_piv

,read,write,metadata
trange,,,
1,39.384121,0.0,157.112274
2,102.217094,0.0,272.024048
3,118.490753,0.0,333.574982
4,128.453979,0.0,358.065979
5,136.352509,0.0,389.777374
...,...,...,...
1106,0.000000,0.0,15.551300
1115,0.000000,0.0,16.820549
1138,0.000000,0.0,14.048192


time: 33.1 ms (started: 2023-01-10 21:07:36 -08:00)


In [31]:
from bokeh.io import show
from bokeh.models import ColumnDataSource
from bokeh.plotting import figure
from bokeh.palettes import Category10_3

symbols = list(trange_agg_cut_piv.index)
impacts = list(trange_agg_cut_piv.columns)

source = ColumnDataSource(trange_agg_cut_piv)

p = figure(width=800, height=200)
p.vbar_stack(impacts, width=0.7, x="trange", source=source, color=Category10_3, legend_label=impacts)

p.y_range.start = 0
p.xgrid.grid_line_color = None
p.legend.location = "top_right"

show(p)

time: 338 ms (started: 2023-01-10 21:00:51 -08:00)


In [39]:
ddf = ddf.persist()

time: 51.7 ms (started: 2023-01-03 00:19:35 -08:00)


In [40]:
hlm_trange = compute_hlm(ddf=ddf, group_by='trange')
hlm_trange

,duration
,sum
npartitions=1,
,float32
,...


time: 101 ms (started: 2023-01-03 00:19:44 -08:00)


In [41]:
hlm_trange.compute()

,duration
,sum
trange,
"(1080000000.0, 1090000000.0]",1721.090942
"(1100000000.0, 1110000000.0]",1690.369751
"(1070000000.0, 1080000000.0]",1677.583862
"(1060000000.0, 1070000000.0]",1543.197388
"(1120000000.0, 1130000000.0]",1475.074829
...,...
"(27410000000.0, 27420000000.0]",0.000000
"(27420000000.0, 27430000000.0]",0.000000


time: 3.9 s (started: 2023-01-03 00:19:45 -08:00)


In [43]:
hlm_trange_filtered = filter_delta_delayed(df=hlm_trange, delta=DELTA)
hlm_trange_filtered.compute()

duration                       
                                     csp     delta          sum
trange                                                         
(1080000000.0, 1090000000.0]    0.012900  0.012900  1721.090942
(1100000000.0, 1110000000.0]    0.025569  0.012669  1690.369751
(1070000000.0, 1080000000.0]    0.038142  0.012574  1677.583862
(1060000000.0, 1070000000.0]    0.049709  0.011566  1543.197388
(1120000000.0, 1130000000.0]    0.060764  0.011056  1475.074829
...                                  ...       ...          ...
(6580000000.0, 6590000000.0]    0.943540  0.000101    13.457269
(5700000000.0, 5710000000.0]    0.943641  0.000101    13.429598
(11020000000.0, 11030000000.0]  0.943742  0.000101    13.426023
(4130000000.0, 4140000000.0]    0.943842  0.000101    13.419438
(3950000000.0, 3960000000.0]    0.943942  0.000100    13.392311

[626 rows x 3 columns]

time: 3.47 s (started: 2023-01-03 00:20:26 -08:00)


In [ ]:
hlm_trange_filtered_file_id = compute_hlm2(ddf=ddf[ddf['trange'].isin(hlm_trange_filtered.index)], group_by='file_id')
hlm_trange_filtered_file_id

In [10]:
stat_agg = ddf \
    .groupby(['io_cat']) \
    .agg({
        'acc_pat': [min, max],
        'app': [nunique()],
        'duration': [sum],
        'file_id': [nunique()],
        'hostname': [nunique()],
        'index': ['count'],
        'proc_id': [nunique()],
        'rank': [nunique()],
        'size': [min, max, 'mean', sum],
    }) \
    .compute()
stat_agg = stat_agg[stat_agg.index.isin([1, 2, 3])]
stat_agg['duration', 'per'] = stat_agg.div(stat_agg['duration', 'sum'].sum(), level=0)['duration', 'sum']
stat_agg['file_id', 'per'] = stat_agg.div(stat_agg['file_id', 'nunique'].max(), level=0)['file_id', 'nunique']
stat_agg['index', 'per'] = stat_agg.div(stat_agg['index', 'count'].sum(), level=0)['index', 'count']
stat_agg['proc_id', 'per'] = stat_agg.div(stat_agg['proc_id', 'nunique'].max(), level=0)['proc_id', 'nunique']
stat_agg['size', 'per'] = stat_agg.div(stat_agg['size', 'sum'].sum(), level=0)['size', 'sum']
stat_agg['bw', 'sum'] = stat_agg['size', 'sum'] / stat_agg['duration', 'sum']
stat_agg['bw', 'per'] = stat_agg.div(stat_agg['bw', 'sum'].sum(), level=0)['bw', 'sum']
stat_agg['xfer', 'max'] = pd.cut(stat_agg['size', 'max'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'min'] = pd.cut(stat_agg['size', 'min'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['xfer', 'mean'] = pd.cut(stat_agg['size', 'mean'], bins=XFER_SIZE_BINS, labels=XFER_SIZE_BIN_LABELS, right=True)
stat_agg['bw', 'sum'] = stat_agg['bw', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB/s")
stat_agg['size', 'sum'] = stat_agg['size', 'sum'].apply(lambda x: format(float(x)/1024.0/1024.0/1024.0, ".2f") + "GB")
stat_agg.drop(columns=[('size', 'max'), ('size', 'min'), ('size', 'mean')], inplace=True)
stat_agg = stat_agg.reindex(sorted(stat_agg.columns), axis=1)
stat_agg


acc_pat         app            bw             duration                 \
           max min nunique           per        sum       per            sum   
io_cat                                                                         
1            0   0       5  9.967778e-01   9.28GB/s  0.058672    7828.180176   
2            0   0       5  3.222178e-03   0.03GB/s  0.026136    3487.186035   
3            1   0       7 -3.179351e-11  -0.00GB/s  0.915191  122106.781250   

         file_id           hostname      index           proc_id            \
         nunique       per  nunique      count       per nunique       per   
io_cat                                                                       
1       10549343  0.496026       32  526812254  0.736567    1764  0.442105   
2       10445412  0.491139       32   10730937  0.015004    1764  0.442105   
3       21267734  1.000000       32  177682895  0.248429    3990  1.000000   

          rank          size               xfer                
       nunique           per         sum    max    mean   min  
io_cat                                                         
1            1  9.985621e-01  72644.43GB  ~16MB  ~256KB  <4KB  
2            1  1.437939e-03    104.61GB  ~16MB   ~16KB  <4KB  
3         1280 -4.968143e-10     -0.00GB   <4KB    <4KB  <4KB

time: 1min 1s (started: 2023-01-11 12:53:21 -08:00)


In [11]:
# importing the modules
from bokeh.plotting import figure, show
from bokeh.palettes import Category10
from bokeh.transform import cumsum
import math

data = stat_agg.reset_index()
data['io_cat'] = ['read', 'write', 'metadata']
data['duration', 'angle'] = data['duration', 'per'] * 2*math.pi
data['duration', 'color'] = Category10[3]
data['duration', 'per_fmt'] = data['duration', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['size', 'angle'] = data['size', 'per'] * 2*math.pi
data['size', 'color'] = Category10[3]
data['size', 'per_fmt'] = data['size', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")
data['index', 'angle'] = data['index', 'per'] * 2*math.pi
data['index', 'color'] = Category10[3]
data['index', 'per_fmt'] = data['index', 'per'].apply(lambda x: format(float(x) * 100, ".2f") + "%")

data

io_cat acc_pat         app            bw             duration  \
                max min nunique           per        sum       per   
0      read       0   0       5  9.967778e-01   9.28GB/s  0.058672   
1     write       0   0       5  3.222178e-03   0.03GB/s  0.026136   
2  metadata       1   0       7 -3.179351e-11  -0.00GB/s  0.915191   

                   file_id            ...  xfer  duration                   \
             sum   nunique       per  ...   min     angle    color per_fmt   
0    7828.180176  10549343  0.496026  ...  <4KB  0.368649  #1f77b4   5.87%   
1    3487.186035  10445412  0.491139  ...  <4KB  0.164220  #ff7f0e   2.61%   
2  122106.781250  21267734  1.000000  ...  <4KB  5.750317  #2ca02c  91.52%   

           size                      index                   
          angle    color per_fmt     angle    color per_fmt  
0  6.274150e+00  #1f77b4  99.86%  4.627990  #1f77b4  73.66%  
1  9.034839e-03  #ff7f0e   0.14%  0.094270  #ff7f0e   1.50%  
2 -3.121577e-09  #2ca02c  -0.00%  1.560925  #2ca02c  24.84%  

[3 rows x 30 columns]

time: 55.6 ms (started: 2023-01-11 12:54:22 -08:00)


In [12]:
from bokeh.layouts import column, row

def pie_chart(by: str):
        p = figure(height=200, width=300, title=f"{by} by I/O category", toolbar_location=None, 
                tooltips=f"@io_cat_: @{by}_per_fmt (x)", x_range=(-0.5, 1.0))

        p.wedge(x=0, y=0, radius=0.4,
                start_angle=cumsum(f'{by}_angle', include_zero=True), end_angle=cumsum(f'{by}_angle'),
                fill_color=f'{by}_color', line_color=None, legend_field='io_cat_', source=data)

        p.axis.axis_label = None
        p.axis.visible = False
        p.grid.grid_line_color = None

        return p

p_dur = pie_chart('duration')
p_ix = pie_chart('index')
p_size = pie_chart('size')

show(row([p_dur, p_ix, p_size]))

time: 354 ms (started: 2023-01-11 12:54:22 -08:00)
